In [1]:
'Hi iam trying to push into github repo'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Load the CSV file
file_path = "soren.csv"  # Ensure this path is correct
data = pd.read_csv(file_path)

# Drop unnecessary columns
data.drop(['Frame', 'Timestamp', 'Average Pressure (mmHg)', 'Minimum Pressure (mmHg)', 
           'Maximum Pressure (mmHg)', 'Standard Pressure Deviation (mmHg)', 
           'Median Pressure (mmHg)', 'Contact Area (m²)', 'Total Area (m²)', 
           'Estimated Force (N)', 'Range Min (mmHg)', 'Range Max (mmHg)'], axis=1, inplace=True)

# Remove rows with all zero values
data = data[(data != 0).any(axis=1)]

# Create a directory to save the images
output_folder = "images_32x32"
os.makedirs(output_folder, exist_ok=True)

# Get the number of rows after filtering
num_rows = data.shape[0]

# Loop through each row in the dataset and save it as a 32x32 image
for i in range(num_rows):
    # Reshape the data into a 32x32 grid
    grid_data = data.iloc[i].values.reshape(32, 32)
    
    # Create a plot without axis
    plt.figure(figsize=(6, 6))
    plt.imshow(grid_data, cmap='viridis', interpolation='none')
    plt.axis('off')  # Remove axis
    
    # Save the image in the output folder
    image_filename = os.path.join(output_folder, f"pressure_image_{i+1}.png")
    plt.savefig(image_filename, bbox_inches='tight', pad_inches=0)
    plt.close()

print(f"Saved {num_rows} images in the '{output_folder}' folder after removing rows with all zero values.")


Saved 510 images in the 'images_32x32' folder after removing rows with all zero values.


In [2]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Load the CSV file
file_path = "soren.csv"  # Ensure this path is correct
data = pd.read_csv(file_path)

# Drop unnecessary columns
columns_to_drop = ['Frame', 'Timestamp', 'Average Pressure (mmHg)', 'Minimum Pressure (mmHg)', 
                   'Maximum Pressure (mmHg)', 'Standard Pressure Deviation (mmHg)', 
                   'Median Pressure (mmHg)', 'Contact Area (m²)', 'Total Area (m²)', 
                   'Estimated Force (N)', 'Range Min (mmHg)', 'Range Max (mmHg)']

# Drop metadata columns if they exist
data.drop(columns=[col for col in columns_to_drop if col in data.columns], axis=1, inplace=True)

# Remove rows where all values are zero
data = data[(data != 0).any(axis=1)]

# Create a directory to save the 6x6 images
output_folder_6x6 = "images_6x6"
os.makedirs(output_folder_6x6, exist_ok=True)

# Get the number of rows after filtering
num_rows = data.shape[0]

# Generate 6 evenly spaced indices from a 32×32 grid
indices = np.linspace(2, 30, 6, dtype=int)  # Choosing 6 evenly spaced points in a 32x32 grid

# Generate (x, y) coordinate pairs for the 6×6 grid
selected_indices = [(indices[j], indices[k]) for j in range(6) for k in range(6)]

# Loop through each row in the dataset and save it as a 6x6 image
for i in range(num_rows):
    # Reshape the data into a 32x32 grid
    grid_data = data.iloc[i].values.reshape(32, 32)
    
    # Extract the 6x6 grid correctly
    reduced_grid_data = np.array([
        [grid_data[x, y] for y in np.linspace(2, 30, 6, dtype=int)]  # Select columns
        for x in np.linspace(2, 30, 6, dtype=int)  # Select rows
    ])
    
    # Create a plot without axis
    plt.figure(figsize=(4, 4))
    plt.imshow(reduced_grid_data, cmap='viridis', interpolation='none')
    plt.axis('off')  # Remove axis
    
    # Save the image in the output folder
    image_filename_6x6 = os.path.join(output_folder_6x6, f"pressure_image_6x6_{i+1}.png")
    plt.savefig(image_filename_6x6, bbox_inches='tight', pad_inches=0)
    plt.close()

print(f"Saved {num_rows} 6x6 images in the '{output_folder_6x6}' folder after processing the original data.")


Saved 510 6x6 images in the 'images_6x6' folder after processing the original data.


In [7]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.models import vgg16, VGG16_Weights
from PIL import Image
import torch.nn.functional as F
import pytorch_msssim  # For SSIM loss

# ===========================
# Device Setup
# ===========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ===========================
# Generator Model
# ===========================
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(channels),
        )

    def forward(self, x):
        return x + self.block(x)

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.initial = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
        )

        self.residuals = nn.Sequential(
            ResidualBlock(64),
            ResidualBlock(64),
            ResidualBlock(64),
            ResidualBlock(64),
        )

        self.upsampling = nn.Sequential(
    nn.ConvTranspose2d(64, 128, kernel_size=4, stride=2, padding=1),  # 6x6 → 12x12
    nn.BatchNorm2d(128),
    nn.ReLU(inplace=True),

    nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),  # 12x12 → 24x24
    nn.BatchNorm2d(64),
    nn.ReLU(inplace=True),

    nn.ConvTranspose2d(64, 32, kernel_size=3, stride=1, padding=1),   # 24x24 → 24x24
    nn.BatchNorm2d(32),
    nn.ReLU(inplace=True),

    # 🔥 Upsample 24x24 → 32x32
    nn.Upsample(size=(32, 32), mode='bilinear', align_corners=True),

    nn.Conv2d(32, 3, kernel_size=3, stride=1, padding=1),
    nn.Tanh()
)


    def forward(self, x):
        x = self.initial(x)
        x = self.residuals(x)
        x = self.upsampling(x)
        return x
    
# ===========================
# Discriminator Model
# ===========================
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# ===========================
# Loss Functions
# ===========================
criterion = nn.BCELoss()

def perceptual_loss(generated, target):
    vgg = vgg16(weights=VGG16_Weights.IMAGENET1K_V1).features[:16].to(device)
    for param in vgg.parameters():
        param.requires_grad = False

    resize = nn.Upsample(size=(224, 224), mode='bilinear', align_corners=True)
    return F.mse_loss(vgg(resize(generated)), vgg(resize(target)))

def laplacian_loss(generated, target):
    laplacian_kernel = torch.tensor([[0, -1, 0], [-1, 4, -1], [0, -1, 0]], dtype=torch.float32)
    laplacian_kernel = laplacian_kernel.unsqueeze(0).unsqueeze(0).to(generated.device)
    laplacian_kernel = laplacian_kernel.repeat(3, 1, 1, 1)

    gen_edges = F.conv2d(generated, laplacian_kernel, padding=1, groups=3)
    target_edges = F.conv2d(target, laplacian_kernel, padding=1, groups=3)

    return F.l1_loss(gen_edges, target_edges)

def adaptive_d_loss(d_real, d_fake, epoch, total_epochs):
    alpha = max(0.1, 1 - (epoch / total_epochs))
    return alpha * d_real + alpha * d_fake

# ===========================
# Data Preparation
# ===========================
def prepare_augmented_low_res(folder_path):
    images = []
    transform = transforms.Compose([
        transforms.Resize((6, 6), interpolation=Image.BICUBIC),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=10),
        transforms.RandomResizedCrop(6, scale=(0.8, 1.0)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    for filename in os.listdir(folder_path):
        if filename.endswith('.png'):
            img = Image.open(os.path.join(folder_path, filename)).convert("RGB")
            img = transform(img)
            images.append(img)
    return torch.stack(images)

def prepare_images(folder_path, size):
    images = []
    transform = transforms.Compose([
        transforms.Resize(size, interpolation=Image.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    for filename in os.listdir(folder_path):
        if filename.endswith('.png'):
            img = Image.open(os.path.join(folder_path, filename)).convert("RGB")
            img = transform(img)
            images.append(img)
    return torch.stack(images)

low_res_images = prepare_augmented_low_res('images_6x6').to(device)
high_res_images = prepare_images('images_32x32', (32, 32)).to(device)

# ===========================
# Training Setup
# ===========================
generator = Generator().to(device)
discriminator = Discriminator().to(device)

optimizer_G = optim.Adam(generator.parameters(), lr=5e-5, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=2e-5, betas=(0.5, 0.999))


# ===========================
# 🧪 Step 1 & 2: Sanity Check Generator Output Shape
# ===========================
with torch.no_grad():
    sample_input = torch.randn(1, 3, 6, 6).to(device)
    sample_output = generator(sample_input)
    print("🧪 Generator test output shape:", sample_output.shape)


# ===========================
# Training Loop
# ===========================
num_epochs = 300
batch_size = 4

for epoch in range(num_epochs):
    for i in range(0, len(low_res_images), batch_size):
        low_res = low_res_images[i:i+batch_size].to(device)
        high_res = high_res_images[i:i+batch_size].to(device)

        # Train Discriminator
        optimizer_D.zero_grad()
        real_output = discriminator(high_res).view(-1)
        d_loss_real = criterion(real_output, torch.ones_like(real_output).to(device))

        fake_image = generator(low_res)
        fake_output = discriminator(fake_image.detach()).view(-1)
        d_loss_fake = criterion(fake_output, torch.zeros_like(fake_output).to(device))

        d_loss = adaptive_d_loss(d_loss_real, d_loss_fake, epoch, num_epochs)
        d_loss.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()
        fake_image = generator(low_res)
        fake_output = discriminator(fake_image).view(-1)

        g_bce_loss = criterion(fake_output, torch.ones_like(fake_output).to(device))
        g_percep_loss = perceptual_loss(fake_image, high_res)
        g_lap_loss = laplacian_loss(fake_image, high_res)

        perceptual_weight = 0.2 + (epoch / num_epochs) * 0.2
        edge_weight = 0.2 - (epoch / num_epochs) * 0.1

        g_loss = g_bce_loss + perceptual_weight * g_percep_loss + edge_weight * g_lap_loss
        g_loss.backward()
        optimizer_G.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")

    if (epoch + 1) % 50 == 0:
        os.makedirs("models", exist_ok=True)
        torch.save(generator.state_dict(), f"models/generator_epoch{epoch+1}.pth")
        torch.save(discriminator.state_dict(), f"models/discriminator_epoch{epoch+1}.pth")
        print(f"✅ Checkpoint saved at epoch {epoch+1}")

# ===========================
# Save Final Models
# ===========================
os.makedirs("models", exist_ok=True)
torch.save(generator.state_dict(), "models/generatorN.pth")
torch.save(discriminator.state_dict(), "models/discriminatorN.pth")
print("✅ Final models saved as generatorN.pth and discriminatorN.pth")

# ===========================
# Export to ONNX
# ===========================
print("📦 Exporting Generator to ONNX format...")

# Create a dummy input of the correct shape (batch_size=1, channels=3, height=6, width=6)
dummy_input = torch.randn(1, 3, 6, 6).to(device)

# Export the model
torch.onnx.export(
    generator,                         # Model
    dummy_input,                       # Example input
    "models/generator.onnx",          # File path
    export_params=True,
    opset_version=11,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
)

print("✅ ONNX model exported successfully to models/generator.onnx")



Using device: cpu
🧪 Generator test output shape: torch.Size([1, 3, 32, 32])
Epoch [1/300], D Loss: 0.5956, G Loss: 1.6196
Epoch [2/300], D Loss: 0.9336, G Loss: 1.2037
Epoch [3/300], D Loss: 0.6760, G Loss: 1.4524
Epoch [4/300], D Loss: 0.5521, G Loss: 1.6713
Epoch [5/300], D Loss: 1.2073, G Loss: 0.9539
Epoch [6/300], D Loss: 0.4250, G Loss: 1.9437
Epoch [7/300], D Loss: 0.4068, G Loss: 2.0700
Epoch [8/300], D Loss: 0.2230, G Loss: 2.4101
Epoch [9/300], D Loss: 0.2417, G Loss: 2.2685
Epoch [10/300], D Loss: 0.2211, G Loss: 2.5474
Epoch [11/300], D Loss: 1.2400, G Loss: 1.0059
Epoch [12/300], D Loss: 1.2736, G Loss: 0.6999
Epoch [13/300], D Loss: 0.7661, G Loss: 1.3744
Epoch [14/300], D Loss: 1.2222, G Loss: 0.7604
Epoch [15/300], D Loss: 0.4822, G Loss: 1.7804
Epoch [16/300], D Loss: 0.1217, G Loss: 3.0829
Epoch [17/300], D Loss: 0.1268, G Loss: 3.2342
Epoch [18/300], D Loss: 0.1366, G Loss: 2.9668
Epoch [19/300], D Loss: 1.1822, G Loss: 1.0458
Epoch [20/300], D Loss: 1.1603, G Loss: 

OnnxExporterError: Module onnx is not installed!

In [9]:
import os
import torch
from PIL import Image
import torchvision.transforms as transforms

# Make sure to define or import your Generator class before using it
# from your_model_file import Generator

# Load Trained Generator
generator = Generator()
generator.load_state_dict(torch.load("models/generatorN.pth"))
generator.eval()

# Define folders
input_folder = "images_6x6"
output_folder = "upscaled_250N_32x32"
os.makedirs(output_folder, exist_ok=True)

# Correct Transformations for low-res image
transform = transforms.Compose([
    transforms.Resize((6, 6)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Match training normalization
])

# Process images and save with new file names
for i, filename in enumerate(os.listdir(input_folder), start=1):
    if filename.endswith(".png"):
        input_path = os.path.join(input_folder, filename)

        # Load and preprocess the image
        test_image = Image.open(input_path).convert("RGB")
        low_res_input = transform(test_image).unsqueeze(0)  # Add batch dimension

        # Generate upscaled 32×32 image
        with torch.no_grad():
            upscaled_image = generator(low_res_input).squeeze(0)  # Remove batch dimension

            # Convert from [-1, 1] to [0, 1]
            upscaled_image = (upscaled_image + 1) / 2

            # Convert tensor to PIL image
            upscaled_image = transforms.ToPILImage()(upscaled_image)
            
            # Create the new file name following the pattern
            new_filename = f"pressure_image_32x32_{i}.png"
            output_path = os.path.join(output_folder, new_filename)
            
            # Save the upscaled image
            upscaled_image.save(output_path)

        print(f"✅ Upscaled and saved: {output_path}")

print("🚀 All images successfully upscaled to upscaled_250N_32×32 and saved!")


✅ Upscaled and saved: upscaled_250N_32x32/pressure_image_32x32_1.png
✅ Upscaled and saved: upscaled_250N_32x32/pressure_image_32x32_2.png
✅ Upscaled and saved: upscaled_250N_32x32/pressure_image_32x32_3.png
✅ Upscaled and saved: upscaled_250N_32x32/pressure_image_32x32_4.png
✅ Upscaled and saved: upscaled_250N_32x32/pressure_image_32x32_5.png
✅ Upscaled and saved: upscaled_250N_32x32/pressure_image_32x32_6.png
✅ Upscaled and saved: upscaled_250N_32x32/pressure_image_32x32_7.png
✅ Upscaled and saved: upscaled_250N_32x32/pressure_image_32x32_8.png
✅ Upscaled and saved: upscaled_250N_32x32/pressure_image_32x32_9.png
✅ Upscaled and saved: upscaled_250N_32x32/pressure_image_32x32_10.png
✅ Upscaled and saved: upscaled_250N_32x32/pressure_image_32x32_11.png
✅ Upscaled and saved: upscaled_250N_32x32/pressure_image_32x32_12.png
✅ Upscaled and saved: upscaled_250N_32x32/pressure_image_32x32_13.png
✅ Upscaled and saved: upscaled_250N_32x32/pressure_image_32x32_14.png
✅ Upscaled and saved: upscale